<a href="https://colab.research.google.com/github/Jack3690/pydol/blob/main/notebooks/Example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install git+https://github.com/Jack3690/pydol/ -q
!pip install astroquery -q

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.4/15.4 MB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 960.9/960.9 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.9/88.9 kB 760.9 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 22.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.5/296.5 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.3/155.3 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 25.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 42.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.2/316.2 kB 16.7 MB/s eta 

# **DOLPHOT Installation**

In [ ]:
!wget http://americano.dolphinsim.com/dolphot/dolphot2.0.tar.gz
!wget http://americano.dolphinsim.com/dolphot/dolphot2.0.NIRCAM.tar.gz
!wget http://americano.dolphinsim.com/dolphot/nircam_F140M.tar.gz
!wget http://americano.dolphinsim.com/dolphot/nircam_F212N.tar.gz

In [ ]:
!tar -xf dolphot2.0.tar.gz
!tar -xf dolphot2.0.NIRCAM.tar.gz
!tar -xf nircam_F140M.tar.gz
!tar -xf nircam_F212N.tar.gz

In [ ]:
with open('/content/dolphot2.0/Makefile') as f:
  dat = f.readlines()
dat[28]= dat[28][1:]

with open('/content/dolphot2.0/Makefile','w') as f:
  f.writelines(dat)

In [ ]:
cd /content/dolphot2.0/

In [ ]:
import os

In [ ]:
os.system('make all')

In [ ]:
cd /content/

In [ ]:
if 'dolphot2.0' not in os.environ['PATH']:
  os.environ['PATH'] += ':/content/dolphot2.0/bin'
  os.environ['PATH'] += ':/content/dolphot2.0/nircam'

# **Imports**

In [ ]:
from glob import glob
import astropy.io.fits as fits

import os
import astroquery
from astroquery.mast import Observations
import pandas as pd

from astropy.table import Table
from astropy.wcs import WCS
import numpy as np
import multiprocessing as mp
import matplotlib.pyplot as plt
import json
from astropy.coordinates import angular_separation
import astropy.units as u
from pydol.pipeline.jwst import jpipe
import multiprocessing as mp

# **Data**

**Observation ID format**

jw\<ppppp>\<ooo><vvv>\_\<gg>\<s>\<aa>\_\<eeeee>(-<"seg"NNN>)\_\<detector>\_\<prodType>.fits

where

ppppp: program ID number

ooo: observation number

vvv: visit number

gg: visit group

s: parallel sequence ID (1=prime, 2-5=parallel)

aa: activity number (base 36)

eeeee: exposure number

segNNN: the text “seg” followed by a three-digit segment number (optional)

detector: detector name (e.g. ‘nrca1’, ‘nrcblong’, ‘mirimage’)

prodType: product type identifier (e.g. ‘uncal’, ‘rate’, ‘cal’)

In [ ]:
#Example data for F212N filter for M82

mast_dir = 'mast:jwst/product' # Download from MAST
data_dir = '/content/data/stage0/'  # save downloaded data
os.makedirs(data_dir, exist_ok=True)

ext = 'uncal'
# JWST images to be analyzed
image_files = []

for i in range(1,10):
    for j in range(1,5):
            for l in ['b']:
                image_files.append( f"jw01701052001_02102_0000{i}_nrc{l}{j}_uncal.fits")

for i in range(10,17):
    for j in range(1,5):
            for l in ['b']:
                image_files.append( f"jw01701052001_02102_000{i}_nrc{l}{j}_uncal.fits")

for image_file in image_files:
    # Download file (if not already downloaded)
    mast_path  = os.path.join(mast_dir, image_file)
    local_path = os.path.join(data_dir, image_file)
    Observations.download_file(mast_path, local_path=local_path)

In [ ]:
#Example data for M82

mast_dir = 'mast:jwst/product' # Download from MAST
data_dir = '/content/photometry/'  # save downloaded data
os.makedirs(data_dir, exist_ok=True)

# JWST images to be analyzed
image_files = [ f"jw01701-o052_t007_nircam_clear-f212n-sub640_i2d.fits"]

for image_file in image_files:
    # Download file (if not already downloaded)
    mast_path  = os.path.join(mast_dir, image_file)
    local_path = os.path.join(data_dir, image_file)
    Observations.download_file(mast_path, local_path=local_path)

# **Pipeline**

In [ ]:
# For reducing load on the RAM. Let's split pipeline into two steps

input_files = glob(f'/content/data/stage0/*b1*')
out_dir = '/content/'
jwst_data = jpipe(input_files, out_dir)
jwst_data()

# **DOLPHOT Photometry**

In [ ]:
if 'dolphot2.0' not in os.environ['PATH']:
  os.environ['PATH'] += ':/content/dolphot2.0/bin'
  os.environ['PATH'] += ':/content/dolphot2.0/nircam'

In [ ]:
crf_files = glob(f'/content/data/stage3/*b1*crf.fits')

In [ ]:
from pydol.photometry import nircam

In [ ]:
nircam.nircam_phot(crf_files,
                   filter='f212n',
                   output_dir='/content/photometry/',
                   drz_path='/content/photometry/jw01701-o052_t007_nircam_clear-f212n-sub640_i2d',
                   cat_name= 'b1')